# Example 1

Our first neural network

Do not forget: Runtime -> Change Runtime Type -> Python3 and GPU

If you want to start locally check this [link](https://pytorch.org/get-started/locally/).

In [0]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Define model

We have three layers (500, 1000, 10). The last one is important, since we have 10 classes we need 10 output neuron.

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 1000)
        self.fc3 = nn.Linear(1000, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)        
        return F.log_softmax(x, dim=1)

# Loading data

We use `DataLoader` and `DataSet` torch util classes.

In [0]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=128, shuffle=True)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=1000, shuffle=True)

# Training and testing loops

Boilerplate code for training and testing (almost never changes).

In [0]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


# Final parameter tuning and training

In [14]:

use_cuda = torch.cuda.is_available()

torch.manual_seed(42)

device = torch.device("cuda" if use_cuda else "cpu")

model = Net().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.5)

# Test before training

test(model, device, test_loader)

for epoch in range(1, 3 + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

torch.save(model.state_dict(),"mnist.pt")
        



Test set: Average loss: 2.2889, Accuracy: 1131/10000 (11%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.287958
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.264152
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.255640
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.193662
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.200459

Test set: Average loss: 0.1724, Accuracy: 9462/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.142564
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.060985
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.145228
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.140892
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.095072

Test set: Average loss: 0.1110, Accuracy: 9640/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.092519
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.226778
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.131233
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.112240
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.038999

Test set: Average loss: 0.0960, Accuracy: 9686/10000 (97%)

